## GBRT

In [3]:
import pandas as pd
import numpy as np

data = pd.read_csv('C:/Users/ssj01/연구실 프로젝트/Data Set/전처리최종이었음좋겠다.csv')

X = data.drop(['한국표준산업분류코드10차(중분류)', '수익률 (1개월)(%)', '뉴스심리지수'], axis=1)
y = data['수익률 (1개월)(%)']
ym = data['yearmonth']
tk = data['Ticker']

In [5]:
# cross_validate 적용
import warnings
warnings.filterwarnings("ignore")

from hyperopt import hp
from hyperopt import STATUS_OK
from hyperopt.early_stop import no_progress_loss

from sklearn.ensemble import GradientBoostingRegressor
from tqdm import tqdm
import joblib

a = '2004|2005|2006|2007|2008|2009|2010|2011|2012|2013|2014|2015|2016|2017|2018|2019|2020'
ya0 = []
tk0 = []
ym0 = []
ya1 = []

for i in tqdm(range(9)): 
    # tX = X[ym.str.contains(a[:24+5*i])]
    # vX = X[ym.str.contains(a[25+5*i:39+5*i])]
    tvX = X[ym.str.contains(a[:39+5*i])]
    tvym = ym[ym.str.contains(a[:39+5*i])]
    oX = X[ym.str.contains(a[40+5*i:44+5*i])]
    # ty = y[ym.str.contains(a[:24+5*i])]
    # vy = y[ym.str.contains(a[25+5*i:39+5*i])]
    tvy = y[ym.str.contains(a[:39+5*i])]
    oy = y[ym.str.contains(a[40+5*i:44+5*i])]
    otk = tk[ym.str.contains(a[40+5*i:44+5*i])]
    oym = ym[ym.str.contains(a[40+5*i:44+5*i])]
    
    # 하이퍼파라미터 검색 공간 설정
    gbrt_search_space = {
        'learning_rate' : hp.choice('learning_rate', [0.00001, 0.0001, 0.001, 0.01, 0.1]),
        'max_depth' : hp.quniform('max_depth', 1, 6, 1),
        'n_estimators' : hp.quniform('n_estimators', 100, 500, 10),
    }
    
    # 목적 함수 정의
    def objective_func(search_space):
        gbrt = GradientBoostingRegressor(
            loss='huber', 
            alpha=0.999, 
            n_estimators = int(search_space['n_estimators']),
            learning_rate = search_space['learning_rate'],
            max_depth = int(search_space['max_depth']),
            random_state = 1
        )
        
        r = 1
        
        # time series cv
        for j in range(3): # 마지막 3년만 valid로 사용
            tX = X[ym.str.contains(a[:24+5*(j+i)])]
            vX = X[ym.str.contains(a[25+5*(j+i):29+5*(j+i)])]
            ty = y[ym.str.contains(a[:24+5*(j+i)])]
            vy = y[ym.str.contains(a[25+5*(j+i):29+5*(j+i)])]
            
            # for문에 필요한 key를 미리 만듦
            ym_temp = vX['yearmonth'].reset_index(drop=True) # t = ym_temp==k를 위해 reset index
            key_yearmonth = dict.fromkeys(ym_temp)
            
            # vy의 index를 reset
            vy.reset_index(drop=True, inplace=True) # r = r*(vy[sort[-d:].index].mean()+100)/100를 위해 reset index
            
            # 년도, 티커 드랍
            tX = tX.drop(['yearmonth', 'Ticker'], axis=1)
            vX = vX.drop(['yearmonth', 'Ticker'], axis=1)
            
            # fitting
            gbrt.fit(tX, ty)
            
            # 수익률 metric 계산           
            y_pred = pd.Series(gbrt.predict(vX)) # sort_values() 사용을 위해 변환
    
            for k in list(key_yearmonth):
                t = ym_temp==k # 해당 month
                d = round(len(y_pred[t])/10)
                sort = y_pred[t].sort_values()
                r = r*(vy[sort[-d:].index].mean()+100)/100
                
        # 새로운 loss 정의
        new_loss = r*(-1)
        
        return {
            'loss':new_loss,
            'status':STATUS_OK
        }
    
    # fmin()을 사용하여 최적 하이퍼 파라미터 찾기
    from hyperopt import fmin, tpe, Trials
    import warnings
    warnings.filterwarnings('ignore')

    trial_val = Trials() # 결과 저장
    
    best = fmin(
        fn=objective_func,
        space=gbrt_search_space,
        algo=tpe.suggest,
        max_evals=1000,    # 최대 반복 횟수 지정
        trials=trial_val,
        return_argmin=False, # hp.choice의 경우 값 대신 index를 반환함 
        early_stop_fn=no_progress_loss(20), # 30회 동안 개선되지 않으면 조기중지
        # https://github.com/hyperopt/hyperopt/issues/689
        rstate=np.random.default_rng(42) # https://github.com/hyperopt/hyperopt/issues/838
    )
    
    # 정수형 하이퍼 파라미터 값이 뒤에 .0이 붙어서 실수형으로 반환됨을 유의하기!
    best['n_estimators'] = int(best['n_estimators'])
    best['max_depth'] = int(best['max_depth'])
    print(best) 
    
    reg0 = GradientBoostingRegressor(
        loss = 'huber', 
        alpha = 0.999,
        random_state = 1,
        **best)
    
    import warnings
    warnings.filterwarnings('ignore')
    
    tvX = tvX.drop(['yearmonth', 'Ticker'], axis=1)
    oX = oX.drop(['yearmonth', 'Ticker'], axis=1)
    
    reg0.fit(tvX, tvy)
    
    yy0 = reg0.predict(oX)
    ya0 = ya0 + oy.tolist()
    ya1 = ya1 + yy0.tolist()
    tk0 = tk0 + otk.tolist()
    ym0 = ym0 + oym.tolist()
    joblib.dump(reg0,'GBRT_bayesian_last3_'+str(i)+'.pkl')
    
GBRT = pd.DataFrame({'real':ya0,'predict':ya1,'ticker':tk0,'yearmonth':ym0})
GBRT.to_csv('C:/Users/ssj01/연구실 프로젝트/Data Set/GBRT_bayesian.csv')    

  0%|                                                                                            | 0/9 [00:00<?, ?it/s]


  6%|██▏                                   | 59/1000 [10:47:09<172:01:34, 658.12s/trial, best loss: -7.188848719651496]
{'learning_rate': 0.1, 'max_depth': 1, 'n_estimators': 130}


 11%|████████▎                                                                  | 1/9 [10:48:21<86:26:48, 38901.10s/it]


  4%|█▌                                    | 42/1000 [10:42:30<244:15:14, 917.86s/trial, best loss: -5.145708507825626]
{'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 490}


 22%|████████████████▋                                                          | 2/9 [21:37:41<75:42:32, 38936.06s/it]

...

 78%|█████████████████████████████████████████████████████████▌                | 7/9 [102:17:32<34:27:02, 62011.49s/it]


  4%|█▌                                  | 43/1000 [20:38:39<459:27:16, 1728.36s/trial, best loss: -2.9775218589230685]
{'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 330}


 89%|█████████████████████████████████████████████████████████████████▊        | 8/9 [123:04:54<18:21:36, 66096.10s/it]


  4%|█▎                                  | 38/1000 [19:37:21<496:45:43, 1858.98s/trial, best loss: -2.0744934164552946]
{'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 230}


100%|█████████████████████████████████████████████████████████████████████████████| 9/9 [142:48:52<00:00, 57125.86s/it]


In [26]:
3.2683060016887064**(1/3)

1.4840239292706257

In [6]:
ya0 = pd.Series(GBRT['real'])
ya1 = pd.Series(GBRT['predict'])
tk0 = pd.Series(GBRT['ticker'])
ym0 = pd.Series(GBRT['yearmonth'])

r1 = []
r2 = []
for i in list(dict.fromkeys(ym0)):
    t = ym0==i
    d = round(len(ya1[t])/10)
    sort = ya1[t].sort_values()
    r1.append(ya0[sort[-d:].index].mean())
    r2.append(ya0[sort[0:d].index].mean())

rr1 = 1
rgbrt1 = [1]
for i in r1:
    rr1 = rr1*(i+100)/100
    rgbrt1.append(rr1)
rr2 = 1
rgbrt2 = [1]
for i in r2:
    rr2 = rr2*(i+100)/100
    rgbrt2.append(rr2)

In [7]:
rgbrt1[-1]

46.907258460843856

In [29]:
r1 = []
for i in list(dict.fromkeys(ym0)):
    t = ym0==i
    d = round(len(ya1[t])/10)
    sort = ya1[t].sort_values()
    r1.append(ya0[sort[-d:].index].mean())
    r2.append(ya0[sort[0:d].index].mean())

In [30]:
for i in range(9):
    rr1 = 1
    for j in r1[0+i*12:12+i*12]:
        rr1 = rr1*(j+100)/100
    print("year:" ,i, "test 수익률:", rr1)

year: 0 test 수익률: 1.5695060270892631
year: 1 test 수익률: 1.5163619763492913
year: 2 test 수익률: 1.6748012404122685
year: 3 test 수익률: 1.936605118472417
year: 4 test 수익률: 1.8758044372256604
year: 5 test 수익률: 1.1977774682310713
year: 6 test 수익률: 1.2145797739127868
year: 7 test 수익률: 1.3032798448185603
year: 8 test 수익률: 1.7086078863043406
